In [1]:
import os, cv2
from ipywidgets import interact, FloatSlider, IntSlider
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
plt.style.use("dark_background")
%matplotlib inline

from sem3d.data import get_imgs, get_keys, get_pairs, get_rectified
from sem3d.rectification import rectify, get_filtered_kp, translation_alignment
from sem3d.plots import plot_epipolar_lines, plot_disp, plot_disp_trends
from sem3d.geometry import center, get_rotation_angles, rotate_point_img, rotate_point, rotate_img

In [2]:
def rotation_alignment(q1, q2, t1, t2):
    """
    """
    q1 = rotate_point(q1, t1)
    q2 = rotate_point(q2, t2)
    q1, q2 = translation_alignment(q1, q2)
    return q1, q2

def translation_alignment(q1, q2, x_margin=2):
    x_shift = (q1 - q2).max(0)[0] + x_margin
    y_shift = (q1 - q2).mean(0)[1]
    q1 -= np.array([[x_shift, y_shift]])
    return q1, q2

def crop_pt_patch(q, corner=(200, 400), size=(400, 50), center=False):
    x,y = corner
    w,h = size

    x_msk = np.logical_and(q[:,0]>x, q[:,0]<x+w)
    y_msk = np.logical_and(q[:,1]>y, q[:,1]<y+h)
    msk = np.logical_and(x_msk, y_msk)
    q = q[msk]
    if center:
        pass
    return q, msk

def crop_img_patch(img, corner=(200, 400), size=(400, 50)):
    x,y = corner
    w,h = size
    return img[y:y+h,  x:x+w]

In [3]:
def correct_point(q, alpha, skew):
    mat = get_intrinsic(alpha, skew)
    if isinstance(q, list):
        return [(mat @ q_.T).T for q_ in q]
    else:
        return (mat @ q.T).T
    
def get_intrinsic(alpha, skew):
    return np.array([[1+alpha, 0], [skew,1]])

In [4]:
imgs = get_imgs("MEC")
keys = get_keys("MEC")

In [5]:
residual_threshold = .1
img1, img2 = imgs[keys[0]], imgs[keys[-1]]
q1, q2 = get_filtered_kp(img1, img2, residual_threshold=residual_threshold)

14410 keypoints matched
10184 keypoints filtered by coef (0.7). 4226 remaings
12 keypoints filtered by distance (100). 4214 remaings
976 keypoints filtered by intensity (50). 3238 remaings
3238 keypoints matched
1262 keypoints filtered by ransac (0.1). 1976 remaings


In [6]:
max_disp = 5
idx = np.random.choice(q1.shape[0], size=10)

Img1, Img2, Q1,Q2 = img1, img2, q1, q2
t1, t2 = get_rotation_angles(q1, q2)
init_angle = t1
init_delta = (t2 - t1)
cmap = matplotlib.cm.get_cmap('jet')

left_slider = FloatSlider(
    value=t1,
    min=-90,
    max=90,
    step=.01,
    description='t1',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.2f'
)

right_slider = FloatSlider(
    value=t2,
    min=-90,
    max=90,
    step=.01,
    description='t2',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.2f'
)

ratio_slider = FloatSlider(
    value=0,
    min=-5,
    max=5,
    step=.01,
    description='alpha',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.2f'
)

skew_slider = FloatSlider(
    value=0,
    min=-5,
    max=5,
    step=.01,
    description='skew',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.2f'
)

@interact(t1=left_slider, t2=right_slider, alpha=ratio_slider, skew=skew_slider)
def visualization(t1, t2, alpha, skew):
    
    fig,ax = plt.subplots(2,4, figsize=(40,20))
    q1, q2 = correct_point([Q1,Q2], alpha, skew)
    q1, q2 = rotation_alignment(q1, q2, t1, t2)
    
    
    x_indx = q1
    color_x = cmap(x_indx[:,0] / x_indx[:,0].max())
    color_y = cmap(x_indx[:,1] / x_indx[:,1].max())
    x_y_axis = (x_indx[:,0], x_indx[:,1])
    x_y_diff = ((q1-q2)[:,0], (q1-q2)[:,1])
    
    ax[0,0].scatter(x_y_axis[0], x_y_diff[0], c=color_y, alpha=.5)
    #ax[0,0].set_ylim(-max_disp, max_disp)
    ax[0,1].scatter(x_y_diff[0], x_y_axis[1], c=color_x, alpha=.5)
    #ax[0,1].set_xlim(-max_disp, max_disp)
    ax[1,0].scatter(x_y_axis[0], x_y_diff[1], c=color_y, alpha=.5)
    ax[1,0].set_ylim(-max_disp, max_disp)
    ax[1,1].scatter(x_y_diff[1], x_y_axis[1], c=color_x, alpha=.5)
    ax[1,1].set_xlim(-max_disp, max_disp)
    
    #ax[0,-1].set_ylim(img1.shape[0], 0)
    _, msk = crop_pt_patch(Q1, corner=(400,0), size=(100,100))
    q2_ = q2[msk]
    q1_ = q1[msk]
    ax[0,-1].scatter(q1_[:,0], q1_[:,1])
    ax[0,-1].scatter(q2_[:,0], q2_[:,1])
    
    q1_, msk = crop_pt_patch(q1, corner=(400, 500), size=(100,100))
    q2_ = q2[msk]
    ax[1,-1].scatter(q1_[:,0], q1_[:,1])
    ax[1,-1].scatter(q2_[:,0], q2_[:,1])

    #ax[0,2].imshow(rotate_img(img1, angle))
    #ax[1,2].imshow(rotate_img(img2, angle))

interactive(children=(FloatSlider(value=-13.397078611924773, description='t1', max=90.0, min=-90.0, step=0.01)…